In [1]:
import { requireCytoscape, requireCarbon, linePlot } from "./lib/draw";

requireCarbon();
requireCytoscape();

# Lecture 9: Problem Solving

## Where Were We?

Taking a break from languages.

1. Language primitives (i.e., building blocks of languages)
2. Language paradigms (i.e., combinations of language primitives)
3. Building a language (i.e., designing your own language)

## Review

- Instead, we will review every language primitive we have learned so far.
- And see how the concept behind each primitive can be used to solve a LeetCode problem.

### Pure vs. impure functions

```
pure(1) = 2
pure(1) = 2
pure(1) = 2
pure(1) = 2
pure(1) = 2
pure(1) = 2

question: what does pure(1) give?

impure(1) = 2
impure(1) = 3
impure(1) = 3
impure(1) = 3
impure(1) = 2
impure(1) = 1
impure(1) = 5

pure(arr) = arr2
pure(arr) = arr2
pure(arr) = arr2
pure(arr) = arr2
pure(arr) = arr2

impure(arr) = arr2
impure(arr) = arr2
impure(arr) = arr3
impure(arr) = arr

question: what does impure(1) give?
```

### First-class Functions

```ts
(x: number) => (y: (z: number) => number) => y(x)

(x: number) => {
  function g(y: (z: number) => number)) {
    return y(x);
  }
  return g;
  // return (y: (z: number) => number) => y(x)
}

function map<T, U>(arr: T[], f: (x: T) => U): U[] {
   ...
}

const arr = [];
for (const x of arr) {
  arr.push(f(x))
}

arr.map(f)


function filter<T>(arr: T[], f: (x: T) => boolean): T[] {
   ...
}

const arr = [];
for (const x of arr) {
  if (f(x)) {
     arr.push(f(x)) 
  }
}

arr.filter(f)


function reduce<T, U>(f: (acc: U, x:T) => U, init: U): U {
   ...
}

let acc = init;
for (const x of arr) {
  acc = f(acc, x);
}

arr.reduce(f, init);

```

In [2]:
const f = (x: number) => x

### Closures

```
onClick(callback);

function codeBlock() {
  let counter = 0;
  function callback(x) {
    .. use x and counter
  }
  return callback;
}

const callback = codeBlock();
```

### Recursion

```ts
function f(x) {
   ... f(smaller x);
}
```

### Recursive Thinking 1

```ts
maxArr([1, 2, 3, 4, 5, 6])
  = max(1, maxArr([2, 3, 4, 5, 6]))
  = max(1, max(2, maxArr([3, 4, 5, 6]))
  = ...
```


```ts
maxArr([1, 2, 3, 4, 5, 6])
  = max(maxArr([1, 2, 3]), maxArr([4, 5, 6]))
  = ...
```  

### Recursive Thinking 2

```ts
sort([1, 2, 3, 4, 5, 6]) = [6,5,4,3,2,1]
merge(sort([1, 2, 3]), sort([4, 5, 6]))
merge([3,2,1], [6,5,4])

sort([1, 2, 3])
   merge(sort([1, 2]), sort([3]))
```

## Algebraic data-types

```ts
type tree<T> = 
    { tag: "LEAF" }
  | { tag: "NODE", contents: T, left: tree<T>, right: tree<T> };
```

## LeetCode Problem 30: Substring with Concatenation of All Words

27.8% Hard

You are given a string s and an array of strings words of the same length.
Return all starting indices of substring(s) in s that is a concatenation of each word
in words exactly once, in any order, and without any intervening characters.

You can return the answer in any order.

### Example 1:

```
Input: s = "barfoothefoobarman", words = ["foo","bar"]
Output: [0,9]
Explanation: Substrings starting at index 0 and 9 are "barfoo" and "foobar" respectively.
The output order does not matter, returning [9,0] is fine too.
```

In [3]:
const s = "barfoothefoobarman";
const words = ["foo","bar"];

### Example 2:

```
Input: s = "wordgoodgoodgoodbestword", words = ["word","good","best","word"]
Output: []
```

In [4]:
const s2 = "wordgoodgoodgoodbestword";
const words2 = ["word","good","best","word"];

### Example 3:

```
Input: s = "barfoofoobarthefoobarman", words = ["bar","foo","the"]
Output: [6,9,12]
```

In [5]:
const s3 = "barfoofoobarthefoobarman";
const words3 = ["bar","foo","the"];

### Example 4:

```
Input: s = "anttan", words = ["ant", "tan"]
Output: [0]
```

### Example 5:

In [6]:
const s4 = "lingmindraboofooowingdingbarrwingmonkeypoundcake";
const words4 = ["fooo","barr","wing","ding","wing"];

## Using Recursive Thinking

- It's tempting to immediately start coding
- But let's think about what are the smaller pieces we can use to build the larger solution from
- For starters, let's just think about how we would simply check that a single solution exists starting from the beginning of the string

### Checking that a single solution exists

#### Example 1

```ts
search1("barfoothefoobarman", ["foo", "bar"])
  = search1("foothefoobarman", ["foo"]);
  = search1("thefoobarman", [])
  = [] === []
```

#### Example 2

```ts
search1(["bar", "foo", "the", "foo", "bar", "man"], ["foo","bar"]);
  = search1(["foo", "the", "foo", "bar", "man"], ["foo"]);
  = search1(["the", "foo", "bar", "man"], []);
  = [] ==== []
```   

#### Example 3

```ts
search1(["bar", ["foo", ["the", ["foo", ["bar", ["man"]]]]]], ["foo","bar"]);
  = search1(["foo", "the", "foo", "bar", "man"], ["foo"]);
  = search1(["the", "foo", "bar", "man"], []);
  = [] === []
```

#### Putting it together?
    
Just traverse the string, adding the locations
    
```ts
entireString(loc, string, arr)
  = concat(entireString(loc + 1, string[1:], arr), findSubstring(string, arr) ? [loc] : [])
  = ...
```

## Coding findSubstring

### Enter ADTs to model data

In [7]:
type lcstr = // leetcode string
    { 
        tag: "LEAF"
    }
|   {
        tag: "WORD",
        word: string,   // the substring
        idx: number,    // the index of ths substring into the original string
        rest: lcstr     // the rest of the leetcode string
};

In [8]:
function mkLeaf(): lcstr {
    return {
        tag: "LEAF"
    };
}

function mkWord(word: string, idx: number, rest: lcstr): lcstr {
    return {
        tag: "WORD",
        word: word,
        idx: idx,
        rest: rest
    };
}

### Convert String into ADT

```ts
stringToLcstr("barfoobar", 3)
  = mkWord("bar", 0, stringToLcstr("foobar", 3))
  = mkWord("bar", 0, mkWord("foo", 3, stringToLcstr("bar", 3)))
  = mkWord("bar", 0, mkWord("foo", 3, mkWord("bar", 6, stringToLcstr("", 3))))
  = mkWord("bar", 0, mkWord("foo", 3, mkWord("bar", 6, mkLeaf()))))
```

In [9]:
function stringToLcstr(s: string, len: number): lcstr {
    function go(s: string, idx: number) {
        if (s.length === 0) {
            return mkLeaf();
        } else {
            // Question: what does len refer to
            return mkWord(s.substring(0, len), idx, go(s.substring(len), idx + len));
        }
    }
    return go(s, 0);
}

In [10]:
const s = "barfoobar";
JSON.stringify(stringToLcstr(s, 3))

{"tag":"WORD","word":"bar","idx":0,"rest":{"tag":"WORD","word":"foo","idx":3,"rest":{"tag":"WORD","word":"bar","idx":6,"rest":{"tag":"LEAF"}}}}


In [11]:
function lcstrToString(s: lcstr) {
    switch (s.tag) {
        case "LEAF": {
            return "";
        }
        case "WORD": {
            return s.word + s.idx + lcstrToString(s.rest);
        }
    }
}

lcstrToString(stringToLcstr(s, 3))

bar0foo3bar6


In [12]:
const s = "barfoothefoobarman";
lcstrToString(stringToLcstr(s, 3))

bar0foo3the6foo9bar12man15


### Back to search1

As a reminder we had

```ts
search1(["bar", "foo", "the", "foo", "bar", "man"], ["foo","bar"]);
  = search1(["foo", "the", "foo", "bar", "man"], ["foo"]);
  = search1(["the", "foo", "bar", "man"], []);
  = [] === []
```

We're writing `["bar", "foo", "the", "foo", "bar", "man"]` for the `lcstr`.


We'll code a modification where we just return the index

```ts
search(0, ["bar", "foo", "the", "foo", "bar", "man"], ["foo","bar"]);
  = search(0, ["foo", "the", "foo", "bar", "man"], ["foo"]);
  = search(0, ["the", "foo", "bar", "man"], []);
  = [] === [] ? 0 : undefined
```

In [13]:
function search(start: number, s: lcstr, words: string[]): number|undefined {
    switch (s.tag) {
        case "LEAF": {
            return words.length === 0 ? start : undefined;
        }
        case "WORD": {
            if (words.length === 0){
                return start;
            } else {
                for (let i = 0; i < words.length; i++) {
                    if (words[i] === s.word) {
                        start = start === undefined ? s.idx : start;
                        // ex smallerWord(["foo", "bar"]) = ["bar"]
                        const smallerWords = words.slice(0, i).concat(words.slice(i + 1));
                        return search(start, s.rest, smallerWords);
                    }
                }
                return undefined;
            }
        }
    }
}

In [14]:
const arr = [1, 2, 3, 4];
const i = 2;
const left = arr.slice(0, i) // [1, 2]
const right = arr.slice(i + 1) // [4]
console.log(left)
console.log(right)
left.concat(right)

[ 1, 2 ]
[ 4 ]
[ 1, 2, 4 ]


In [15]:
const s = "barfoothefoobarman";
const words = ["foo", "bar"];
search(9, stringToLcstr(s, 3), words)

9


In [16]:
// ["bar", "foo", "the", "foo", "bar", "man"]
function entireString(acc: number[], lc: lcstr): number[] {
    switch (lc.tag) {
        case "LEAF": {
            return acc;
        }
        case "WORD": {
            const res = search(undefined, lc, words);
            if (res !== undefined) {
                acc.push(res);
                return entireString(acc, lc.rest);
            } else {
                return entireString(acc, lc.rest);
            }
        }
    }
}


### Putting it together

In [17]:
function findSubstring(s: string, words: string[]): number[] {
    if (words.length === 0) {
        return [];
    }

    // 1. Create a model
    type lcstr = 
        { tag: "LEAF" }
      | { tag: "WORD", word: string, idx: number, rest: lcstr };

    function Leaf(): lcstr {
        return { tag: "LEAF" };
    }

    function Word(word: string, idx: number, rest: lcstr): lcstr {
        return { tag: "WORD", word: word, idx: idx, rest: rest };
    }
    
    // 2. Solved a reduced problem, i.e., just one index
    function search(start: number, s: lcstr, words: string[]) {
        switch (s.tag) {
            case "LEAF": {
                return words.length === 0 ? start : undefined;
            }
            case "WORD": {
                if (words.length === 0){
                    return start;
                } else {
                    // Reminder: we can search any order of the strings!
                    for (let i = 0; i < words.length; i++) {
                        if (words[i] === s.word) {
                            start = start === undefined ? s.idx : start;
                            return search(start, s.rest, words.slice(0, i).concat(words.slice(i + 1)));
                        }
                    }
                    return undefined;
                }
            }
        }
    }
    
    // 3. Used the reduced problem to solve the actual problem, i.e., all indices
    function entireString(acc: number[], lc: lcstr): number[] {
        switch (lc.tag) {
            case "LEAF": {
                return acc;
            }
            case "WORD": {
                const res = search(undefined, lc, words);
                if (res !== undefined) {
                    return entireString(acc.concat(res), lc.rest);
                } else {
                    return entireString(acc.concat(res), lc.rest);
                }
            }
        }
    }
    
    // 4. Bit contrived
    const acc: number[][] = [];
    for (let i = 0; i < words[0].length; i++) {
        // i = 0 "go(barfoofoobarthefoobarman, ["bar","foo","the"])";
        // i = 1 "go(arfoofoobarthefoobarman, ["bar","foo","the"])";
        // i = 2 "go(rfoofoobarthefoobarman)";
        acc.push(entireString([], stringToLcstr(s.substring(i), words[0].length)).map((x: number) => x + i))
    }
    
    return acc.reduce((acc, x) => acc.concat(x));
}

In [18]:
function timeFunction(name, f) {
    console.log(`--------------------------`);
    console.log(`${name} started..`);
    const t0 = process.hrtime()
    f();
    const t1 = process.hrtime(t0);
    console.log(`${f.name} completed..`);
    console.info('Execution time (hr): %ds %dms', t1[0], t1[1] / 1000000);
    return t1[0] + t1[1] / 1000000 / 1000;
}


In [19]:
const counts = [];
const times = [];
for (let i = 1; i < 10; i++) {
    let s = "";
    const words = [];
    for (let j = 0; j < i*10; j++) {
        s += "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa";
        for (let k = 0; k < s.length; k++) {
            words.push("a");
        }
        
    }
    counts.push(s.length);
    times.push(timeFunction("findSubstring", () => findSubstring(s3, words)));
}

linePlot(counts, [times])

--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 1.70325ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 4.733458ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 2.594959ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 4.575583ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 8.022125ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 12.553791ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 14.120125ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 19.059541ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 24.96275ms


## Challenge: make it faster!

1. Do we really need to create `lcstr`?
2. Can we convert the recursion to an iteration?
3. Examine search1. Did we search the same sub-problem multiple times? Hint: dynamic programming